In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, WeekdayLocator,DayLocator, MONDAY
from mpl_finance import  candlestick_ohlc
import time
from IPython import display

#,quotes_historical_yahoo
#print(dir(mpl_finance))
#MAIN SYMBOL CLASS HOLD ALL SYMBOL RATES
class CSymbol():                
    def __init__(self,specs={}):
                
        #get symbols specifications like swaps min volum max volum symbol name precision contract volum etc..
        self.specs=specs            
        self.rates=pd.read_csv(specs["file_name"],sep="\t") 
        
        ##DELETE THIS SECTION
        #this just for debug i forced delete some items to reproduce syncronization issues
        if specs["symbol_name"]=="GBPUSD":
            self.rates.drop(0,inplace=True)
            
        if specs["symbol_name"]=="USDJPY":
            self.rates.drop(2,inplace=True)
            
        if specs["symbol_name"]=="EURUSD":
            self.rates.drop(3,inplace=True)            
        ###END DELETE
        
        self.rates.columns=specs["symbol_name"]+self.rates.columns        
        self.rates = self.rates.rename(columns={(specs['symbol_name']+"Datetime"):"time"})                
        self.rates.set_index("time",inplace=True)
        
        return None
    
    #TODO if we whant a live reader enviroment 
    def updaterates():
        return None
    
#HELPING OPERATION CLAS RECIBE SIMULATOR AND PERFORM OPERATIONS OVER SIMULATOR STATE
class CTrade():
    
    def __init__(self,simulator):
        self.simulator=simulator
        return None
    def openlong(self,symbol,volum):
        return None
    def openshort(self,symbol,volum):
        return None
#HELP POSITIONINFO     
class CPositionInfo():
    
    
    def __init__(self,volumn,xtype,symbol,step_open):
        self.volumn=0.
        self.type=-1 #0 short  1 long
        self.symbol = None#class symbol
        self.step_open =None#step open price
        self.step_close = None #none or step to close price
        self.current_profit = 0. #positive or nevative profit
        
        return None
    
    
#define custom strategy...
class CStrategy():    
    def __init__(self,name,simulator,model):
        #recibe all inputs parameter needed on run_strategy
        self.name="Custom strategy name"
        self.simulator=simulator
        return None
    
    def run_strategy(self,model=None):    
        while self.simulator.Turn():
            #do somthing indicator working then
            operation_list=[]
            simulator.Perform_OPS(Operations_list)            
        return None
    
#same as CStrategy but multiple strategy with shared simulator    


class CMultiStrategy():
    def __init__(self,strategy_list,simulator):
        return None

    
###YOU CAN DEFINE YOUR CUSTOM INDICATORS 
#you need define mandatory fields buffers, buffer and internalsteps
#you need implement you custom metohd calculation(self,step,symbol) is called from simulator enviroment

class CCustomIndicatorMA():
    def __init__(self,name="IndicatorName",specs={}):        
        self.name=name
        self.specs=specs
        self.buffers=1#this indicator implement only one bufer
        self.EMPTY_VALUE=1
        self.buffer=np.zeros((1,self.buffers)) #prepare numpy arry with shape(size initial 1, buffer_size)
        self.buffer[0,0]=self.EMPTY_VALUE
        self.internalsteps=0
        return None
    def calculation(self,step,symbols=[]):
        #CUSTOM CALCULATION FOR INDICATOR calculation method called from simulator.Turn()
        #info this indicator perform a basic MA calculation at price close
        self.internalsteps+=1
        self.buffer=np.resize(self.buffer,self.buffer.shape[0]+1)
        if self.internalsteps>self.specs["period"]:
            #if we can calculate indicator we done            
            #we symbol by symbol_name of this indicator
            rates=None
            symbol=None
            for x in symbols:
                if x.specs["symbol_name"]==self.specs["symbol"]:
                    rates=x.rates                    
                    break
            self.buffer[self.internalsteps]=np.sum(rates[self.specs["symbol"]+"Close"][step-self.specs["period"]:step])/self.specs["period"]            
        else:
            self.buffer[self.internalsteps]=self.EMPTY_VALUE
                    
        return None
 
    
    
    
#example maoscilator reusing MAIndicator calculations    
class CCustomIndicatorMAOscilator():
    def __init__(self,name="IndicatorName",specs={}):        
        self.name=name
        self.specs=specs
        self.buffers=1#this indicator implement only one bufer         
        
        self.subind_fast=CCustomIndicatorMA("temp0",{"symbol":specs["symbol"],"period":specs["period_fast"]})
        self.subind_slow=CCustomIndicatorMA("temp0",{"symbol":specs["symbol"],"period":specs["period_slow"]})        
        self.EMPTY_VALUE=1
        self.buffer=np.zeros((1,self.buffers)) #prepare numpy arry with shape(size initial 1, buffer_size)
        self.buffer[0,0]=self.EMPTY_VALUE
        self.internalsteps=0
        return None
    def calculation(self,step,symbols=[]):   
        #calculate intermediate buffers first
        self.subind_fast.calculation(step,symbols)
        self.subind_slow.calculation(step,symbols)
        
        self.internalsteps+=1
        self.buffer=np.resize(self.buffer,self.buffer.shape[0]+1)
        if self.internalsteps>self.specs["period_slow"]:
            #if we can calculate indicator we done            
            
            self.buffer[self.internalsteps]=self.subind_fast.buffer[self.internalsteps]/self.subind_slow.buffer[self.internalsteps]
            
        else:
            self.buffer[self.internalsteps]=self.EMPTY_VALUE
                    
        return None
 
#reuse code from MAscilator to generate a custom indicator with multiple buffer at once
#{"symbol":specs["symbol_"+x],"period_fast":specs["period_fast"],"period_slow":specs["period_slow"]} ))
class CCustomIndicatorMultiMAOscilator():
    def __init__(self,name="Multioutputs",specs={}):
        self.name=name
        self.specs=specs
        self.EMPTY_VALUE=1
        
        self.buffers=specs["num_outs"]
        self.buffer=np.zeros((1,self.buffers)) #prepare numpy arry with shape(size initial 1, buffer_size)
        
        self.sub_ind_buf=[]
        for x in range(self.buffers):            
            self.sub_ind_buf.append(CCustomIndicatorMAOscilator("tmp", {"symbol":specs["symbol_"+str(x)],"period_fast":specs["period_fast"],"period_slow":specs["period_slow"]} ))
            self.buffer[0,x]=self.EMPTY_VALUE                    
        
        self.internalsteps=0            
        return None
    def calculation(self,step,symbols=[]):
        self.internalsteps+=1
        self.buffer=np.resize(self.buffer,(self.buffer.shape[0]+1,self.buffers))
        #calculate intermediate indicators buffers first
        for x in range(self.buffers):
            self.sub_ind_buf[x].calculation(step,symbols)
            
        if self.internalsteps>self.specs["period_slow"]:
            #if we can calculate indicator we done            
            for x in range(self.buffers):                
                self.buffer[self.internalsteps,x]=self.sub_ind_buf[x].buffer[self.internalsteps]
                
          # print(self.buffer[self.internalsteps])
        else:
            for x in range(self.buffers):
                self.buffer[self.internalsteps,x]=self.EMPTY_VALUE
            
        return None

    
####CLASS CSimulator    
class CSimulator:    
    
    def __init__(self,enviroment,i_symbols=[],indicators=[],inital_step=0):        
        self.Symbols=[] 
        self.Indicators=indicators #list of indicators each perform method calculate current_state
        self.PosInfo=[]#list of active positions
        self.PosHist=[]#history all closed positions
        self.StatsHist=[]#history all stats
        self.enviroment=enviroment
        #basic stats
        self.initial_balance=enviroment["balance"]
        self.current_balance=self.initial_balance
        self.current_equity=self.initial_balance
        #TODO max min equity balance max min equi balanc drwadown etc..
        
        for x in i_symbols:
            newsym=CSymbol(x)
            self.Symbols.append(newsym)
            
        
        self.syncronize_symbols()#syncronize all symbols indexs
        self.set_cursor(inital_step)#init cursos at step 0
        return None
    
    #Syncronize all symbols datetimes by key time
    def syncronize_symbols(self):
        newindex=""
        for x in range(len(self.Symbols)-1):
            newindex= self.Symbols[x].rates.index.union(self.Symbols[x+1].rates.index)

        for x in range(len(self.Symbols)):
            self.Symbols[x].rates=self.Symbols[x].rates.reindex(newindex)
            
        for x in range(len(self.Symbols)):            
            self.Symbols[x].rates.fillna(method='ffill',inplace=True)
           
        
        #once we filled all na values with previous value only na values apear if first time step is na
        #we delete this time step and previus for all time series
        natimes=[]
        for x in range(len(self.Symbols)):
            natimes.append(self.Symbols[x].rates[self.Symbols[x].rates.isnull().any(axis=1)].index.tolist() )
                
        
        maxtimevalid=max(natimes)[0]
        #remove initial invalid times to sicronize all symbols at same time
        for x in range(len(self.Symbols)):
            tmp=self.Symbols[x].rates.reset_index()
            self.Symbols[x].rates=tmp[tmp["time"]>maxtimevalid]
            self.Symbols[x].rates.set_index("time",inplace=True)
                
        return None
    
    def plotSimulator(self,indicators=False,Stats=False,ActiveOrders=False,HistoricOrders=False):
        #for each symbol on Symbols plot until cursor
        mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
        alldays = DayLocator()              # minor ticks on the days
        weekFormatter = DateFormatter('%b %d')  # e.g., Jan 12
        dayFormatter = DateFormatter('%d')      # e.g., 12
        MAXCOLUMNS=2
        rows=int(len(self.Symbols)/MAXCOLUMNS)
        if rows<len(self.Symbols)/MAXCOLUMNS:
            rows+=1
       
        #adebug quotes format        
        display.clear_output(wait=True)
        fig, ax = plt.subplots(rows,MAXCOLUMNS,figsize=(13,13))
        fig.subplots_adjust(bottom=0.2)
        idx=0
        for x in range(rows):
            for y in range(MAXCOLUMNS):           
                if idx<len(self.Symbols):
                    ax[x][y].xaxis.set_major_locator(mondays)
                    ax[x][y].xaxis.set_minor_locator(alldays)
                    ax[x][y].xaxis.set_major_formatter(weekFormatter)
        #ax.xaxis.set_minor_formatter(dayFormatter)
                
        
                    tmp=self.Symbols[idx].rates.reset_index()
        
                    quotes= [tuple([tmp["time"][i],
                             tmp[self.Symbols[idx].specs["symbol_name"]+"Open"][i],
                             tmp[self.Symbols[idx].specs["symbol_name"]+"High"][i],
                             tmp[self.Symbols[idx].specs["symbol_name"]+"Low"][i],
                             tmp[self.Symbols[idx].specs["symbol_name"]+"Close"][i]]) 
                                             for i in range(len(self.Symbols[idx].rates[0:self.current_step]))] 
                
        #time, open, high, low, close, …
                    candlestick_ohlc(ax[x][y], quotes, width=0.6)
        
                    ax[x][y].xaxis_date()
                    ax[x][y].autoscale_view()
                idx+=1
                
        #TODO plot indicators
        #TODO plot historic order
        #TODO plot Active Orders
        #TODO plot actual stats
        plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
        display.display(plt.show())
        return None
    
    #if we whant interactive session we set or initialize cursor at step we needed
    def set_cursor(self,step=0):
        self.current_step=step
        
        return None
    
    #for interactive session return true while step < len(rates)
    #example usage while simulator.Turn():
    #                    perform_ops...    
    def Turn(self):
        
        if self.current_step<len(self.Symbols[0].rates):                
            for x in range(len(self.Indicators)):
                self.Indicators[x].calculation(self.current_step,self.Symbols)
                
            self.calcSates()
            self.set_cursor(self.current_step+1)
            
            return True
        return False
    
    #use together with Turn
    def Perform_OPS(self,operation):
        return None
    #return n pandas frames with all Symbols sycronizeds
    def getRatesHist(self):
        ret=pd.DataFrame()
        rates_list=[]
        indicators_list=[]
        #return rates in joined pandas and list of pandas for easy processing latter
        for x in range(len(self.Symbols)):
            ret=pd.concat((ret,self.Symbols[x].rates[0:self.current_step]),axis=1)
            rates_list.append(self.Symbols[x].rates[0:self.current_step])
            
        #return all indicators buffers in list of bufer with shape(steps_procesed,buffer_len)
        for x in range(len(self.Indicators)):
            indicators_list.append(self.Indicators[x].buffer)
        return ret,rates_list,indicators_list
    
    #return current stadisitical opbjects
    def getStats(self):
        return None
    
    #return all current stadistical object from step 0 to current state
    def getStatsHistory(self):
        return None
    
    #private method called at end of turn
    def calcSates(self):
        return None
    
    #return all active positions
    def PositionsCount(self):
        return None
    

    


In [26]:

def main():
    print("Init")
    ##ADD SYMBOLS TO SIMULATOR    
    all_symbols=[]
    specs={"file_name":'data/EURUSD0.csv',
           "symbol_name":"EURUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    all_symbols.append(specs)
    
    specs={"file_name":'data/GBPUSD0.csv',
           "symbol_name":"GBPUSD",
           "precision":5,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
            
    
    all_symbols.append(specs)
    
    specs={"file_name":'data/USDJPY0.csv',
           "symbol_name":"USDJPY",
           "precision":3,
           "swap_long":-0.8,
           "swap_short":-0.4,
           "contract_volum":100000}
    
    all_symbols.append(specs)
    
    #ADD INDICATORS TO SIMULATOR
    indicator_list=[]
        
    #Simple MA Indicator for EURUSD
    specs={"symbol":"EURUSD","period":5}
    eurusd_ma_fast=CCustomIndicatorMA("EURUSD_Fast_ma",specs)
    
    
    #Simple MA Indicator for GBPUSD
    specs={"symbol":"GBPUSD","period":5}
    gbpusd_ma_fast=CCustomIndicatorMA("GBPUSD_Fast_ma",specs)
    
    #Simple MA Indicator for USDJPY
    specs={"symbol":"USDJPY","period":15}
    usdjpy_ma_slow=CCustomIndicatorMA("USDJPY_Slow_ma",specs)
    
    
    #MA Osciliator example reusing code for MA Indicator
    specs={"symbol":"EURUSD","period_fast":5,"period_slow":15}    
    eurusd_oscilator=CCustomIndicatorMAOscilator("MAOscilator",specs)
        

    #Multiple outputs or multi symbol indicator reusing MAOscilator
    specs={"num_outs":3,"symbol_0":"EURUSD","symbol_1":"USDJPY","symbol_2":"GBPUSD",
             "period_fast":5,"period_slow":15}
    multisymbol_multiouputs=CCustomIndicatorMultiMAOscilator("Multioutputs",specs)
    
    #MA
    indicator_list.append(eurusd_ma_fast)
    indicator_list.append(gbpusd_ma_fast)
    indicator_list.append(usdjpy_ma_slow)
    
    #MA oscilator
    indicator_list.append(eurusd_oscilator)
    
    #Multi outputs
    indicator_list.append(multisymbol_multiouputs)
    
    
    #DEFINE INITIAL ENVIROMENT FOR SIMULATOR
    initial_step=0
    enviroment={"balance":10000}
    
    
    #create enviroment with symbols indicator and initial balance and initial step
    simulator=CSimulator(enviroment,all_symbols,indicator_list,initial_step)
    
    
    
    #Example Usage 1 simple ploting rates + indicators
  #  while simulator.Turn():
   #     simulator.plotSimulator(indicators=True,Stats=False,ActiveOrders=False,HistoricOrders=False)
    #    time.sleep(0.2)
        
        
    #Example Usage2 run simulator 70% values train model and test strategy with final values
    #1.-Get rate values and indicator values at current state if any
    #2.-Prepare your learning model
    #3.-Define you strategy class with model and trade
        #Try with train data and test data in separated way
    size=len(simulator.Symbols[0].rates)
    train_size=int(size*0.7)
    test_size=size-train_size
    for x in range(train_size):
        simulator.Turn() #turn simulator including indicators calculations if any
    
    #return all rates and indicators values synconized in pandas tabular or list_mode independent plus indicator_vals   
    x_train,list_mode,indicator_vals=simulator.getRatesHist() 
    print(x_train.shape,len(list_mode),len(indicator_vals))
    print(list_mode[0].shape)
    print(indicator_vals[4].shape)
    ####DO YOUR LEARNING MODEL
    ####DO YOUR strategy can test into train data or testing data
    
    
    #Example usage3 in neat or genetics optimization
    #for x in range(generations):
    #   run_specie_in_parallel(CStrategy strategy(simulator,species_genes)):
    #   wait until_all_species_end
    #   mutate_crossover_evolve
    #
    #Finaly we can test with best network strategy(best_genes).run
            
        
    #Example usage4 with q-learning
    #while continue_train==True:
    #   simulator.reinit_state(0)
    #   while simulator.Turn():
    #      enviroment=simulator.get_enviroment()
    #      actions=qlearning(enviroment)
    #      simulator.perform(actions)
    #      rewards=get_rewards(simulator.stats())
    #      
    
    
    
    
    

In [27]:
main()


Init
(1399, 15) 3 5
(1399, 5)
(1400, 3)
